# ਸੈਮੈਂਟਿਕ ਕਰਨਲ

ਇਸ ਕੋਡ ਨਮੂਨੇ ਵਿੱਚ, ਤੁਸੀਂ [ਸੈਮੈਂਟਿਕ ਕਰਨਲ](https://aka.ms/ai-agents-beginners/semantic-kernel) AI ਫਰੇਮਵਰਕ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਇੱਕ ਬੁਨਿਆਦੀ ਏਜੰਟ ਬਣਾਉਗੇ।

ਇਸ ਨਮੂਨੇ ਦਾ ਉਦੇਸ਼ ਤੁਹਾਨੂੰ ਉਹ ਕਦਮ ਦਿਖਾਉਣਾ ਹੈ ਜੋ ਅਸੀਂ ਬਾਅਦ ਵਿੱਚ ਵੱਖ-ਵੱਖ ਏਜੰਟਿਕ ਪੈਟਰਨਾਂ ਨੂੰ ਲਾਗੂ ਕਰਦੇ ਸਮੇਂ ਵਾਧੂ ਕੋਡ ਨਮੂਨਿਆਂ ਵਿੱਚ ਵਰਤਾਂਗੇ।


## ਲੋੜੀਂਦੇ ਪਾਇਥਨ ਪੈਕੇਜਾਂ ਨੂੰ ਇੰਪੋਰਟ ਕਰੋ


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## ਕਲਾਇੰਟ ਬਣਾਉਣਾ

ਇਸ ਉਦਾਹਰਨ ਵਿੱਚ, ਅਸੀਂ [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ਨੂੰ LLM ਤੱਕ ਪਹੁੰਚ ਲਈ ਵਰਤਾਂਗੇ।

`ai_model_id` ਨੂੰ `gpt-4o-mini` ਵਜੋਂ ਪਰਿਭਾਸ਼ਿਤ ਕੀਤਾ ਗਿਆ ਹੈ। ਨਤੀਜੇ ਵੱਖਰੇ ਦੇਖਣ ਲਈ, GitHub Models ਮਾਰਕੀਟਪਲੇਸ ਵਿੱਚ ਉਪਲਬਧ ਕਿਸੇ ਹੋਰ ਮਾਡਲ ਨਾਲ ਮਾਡਲ ਬਦਲਣ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰੋ।

`Azure Inference SDK` ਵਰਤਣ ਲਈ, ਜੋ GitHub Models ਲਈ `base_url` ਲਈ ਵਰਤਿਆ ਜਾਂਦਾ ਹੈ, ਅਸੀਂ Semantic Kernel ਦੇ ਅੰਦਰ `OpenAIChatCompletion` ਕਨੈਕਟਰ ਦੀ ਵਰਤੋਂ ਕਰਾਂਗੇ। ਹੋਰ [ਉਪਲਬਧ ਕਨੈਕਟਰ](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) ਵੀ ਹਨ ਜੋ ਹੋਰ ਮਾਡਲ ਪ੍ਰਦਾਤਾਵਾਂ ਲਈ Semantic Kernel ਦੀ ਵਰਤੋਂ ਕਰਨ ਦੀ ਆਗਿਆ ਦਿੰਦੇ ਹਨ।


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## ਏਜੰਟ ਬਣਾਉਣਾ

ਹੇਠਾਂ ਅਸੀਂ `TravelAgent` ਨਾਮਕ ਏਜੰਟ ਬਣਾਉਂਦੇ ਹਾਂ।

ਇਸ ਉਦਾਹਰਨ ਵਿੱਚ, ਅਸੀਂ ਬਹੁਤ ਸਧਾਰਣ ਹਦਾਇਤਾਂ ਵਰਤ ਰਹੇ ਹਾਂ। ਤੁਸੀਂ ਇਹ ਹਦਾਇਤਾਂ ਬਦਲ ਸਕਦੇ ਹੋ ਤਾਂ ਜੋ ਵੇਖ ਸਕੋ ਕਿ ਏਜੰਟ ਵੱਖ-ਵੱਖ ਤਰੀਕਿਆਂ ਨਾਲ ਕਿਵੇਂ ਪ੍ਰਤੀਕਿਰਿਆ ਕਰਦਾ ਹੈ।


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## ਏਜੰਟ ਚਲਾਉਣਾ

ਹੁਣ ਅਸੀਂ `ChatHistoryAgentThread` ਕਿਸਮ ਦੇ ਥ੍ਰੈਡ ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰਕੇ ਏਜੰਟ ਚਲਾ ਸਕਦੇ ਹਾਂ। ਕੋਈ ਵੀ ਲੋੜੀਂਦੇ ਸਿਸਟਮ ਸੁਨੇਹੇ ਏਜੰਟ ਦੇ invoke_stream ਦੇ `messages` ਕੀਵਰਡ ਆਰਗੂਮੈਂਟ ਨੂੰ ਪ੍ਰਦਾਨ ਕੀਤੇ ਜਾਂਦੇ ਹਨ।

ਇਹਨਾਂ ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰਨ ਤੋਂ ਬਾਅਦ, ਅਸੀਂ ਇੱਕ `user_inputs` ਬਣਾਉਂਦੇ ਹਾਂ ਜੋ ਯੂਜ਼ਰ ਵੱਲੋਂ ਏਜੰਟ ਨੂੰ ਭੇਜਿਆ ਜਾਣ ਵਾਲਾ ਸੁਨੇਹਾ ਹੁੰਦਾ ਹੈ। ਇਸ ਮਾਮਲੇ ਵਿੱਚ, ਅਸੀਂ ਇਸ ਸੁਨੇਹੇ ਨੂੰ `Plan me a sunny vacation` ਤੇ ਸੈਟ ਕੀਤਾ ਹੈ।

ਤੁਸੀਂ ਇਸ ਸੁਨੇਹੇ ਨੂੰ ਬਦਲ ਸਕਦੇ ਹੋ ਤਾਂ ਜੋ ਵੇਖ ਸਕੋ ਕਿ ਏਜੰਟ ਵੱਖ-ਵੱਖ ਤਰੀਕਿਆਂ ਨਾਲ ਕਿਵੇਂ ਪ੍ਰਤੀਕਿਰਿਆ ਕਰਦਾ ਹੈ।


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**ਅਸਵੀਕਰਤੀ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦਾ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁੱਛਤਾਵਾਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਪ੍ਰਮਾਣਿਕ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।  
